# 협업필터링(CF) 추천 - KNN

In [1]:
import numpy as np
print(f'NumPy v{np.__version__}')

import pandas as pd
print(f'pandas v{pd.__version__}')

import sklearn
print(f'scikit-learn v{sklearn.__version__}')

# Only for specifying versions
import sys; print(f'Python v{sys.version}')

NumPy v1.25.0
pandas v1.5.3
scikit-learn v1.2.0
Python v3.9.16 (main, May 17 2023, 17:49:16) [MSC v.1916 64 bit (AMD64)]


## 3.4 이웃을 고려한 CF

앞에서 설명한 단순한 CF 알고리즘을 개선할 수 있는 한 가지 방법은 사용자 중에서 유사도가 높은 사용자를 선정해서 그 사람들의 평점만 가지고 예측을 하는 것이다. 즉 이웃(neighbor)을 전체 사용자로 하는 대신에 유사도가 높은 사람만을 이웃으로 선정해서 이웃의 크기를 줄이는 것이다. 이렇게 대상 사용자와 유사도가 높은 사람의 평가만 사용하면 당연히 예측의 정확도가 올라갈 것으로 예상해 볼 수 있다.

또 하나 고려할 사항은 이웃을 정하는 기준이다.
- KNN: 이웃의 크기를 미리 정해놓고 추천 대상 사용자와 가장 유사한 K명을 선택
- Thresholding: 크기 대신 유사도의 기준(예를 들면 corr coeff 0.8 이상)을 정해놓고 이 기준을 충족시키는 사용자를 이웃으로 정함

일반적으로는 Thresholding이 KNN보다 정확하지만 정해진 기준을 넘는 사용자가 없어서 추천을 하지 못하는 경우가 많기 때문에 KNN이 무난하게 많이 쓰인다.

아래의 2~3번째 cell은 `3-1.ipynb`의 2~3번째 cell과 동일한 코드이다. 다만, `score()`에 새로운 파라미터(`neighbor_size`)가 추가되었다.

In [2]:
# 데이터 읽어 오기
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('../Data/u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# timestamp 제거 
ratings.drop('timestamp', axis='columns', inplace=True)

# train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
u_id = rating_matrix.index
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=u_id, columns=u_id)

In [3]:
# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [4]:
# Neighbor size를 정해서 예측치를 계산하는 함수 
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()

        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()

        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index

        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings.drop(none_rating_idx, inplace=True)       # movie_ratings.dropna(inplace=True)와 같다.
        # 현재 영화를 평가하지 않은 사용자와의 similarity값 제거
        sim_scores.drop(none_rating_idx, inplace=True)

        if neighbor_size == 0:  # Neighbor size가 지정되지 않은 경우
            # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
            if sim_scores.sum() != 0:
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = movie_ratings.mean()
        
        else:                   # Neighbor size가 지정된 경우
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1: 
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = sim_scores.to_numpy()
                movie_ratings = movie_ratings.to_numpy()

                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                selected_idx = user_idx[-neighbor_size:]

                # 유사도를 neighbor size만큼 받기
                sim_scores = sim_scores[selected_idx]
                # 영화 rating을 neighbor size만큼 받기
                movie_ratings = movie_ratings[selected_idx]
                
                # 최종 예측값 계산 
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(cf_knn, neighbor_size=30)

1.0169668415833513

RMSE가 1.017로 이웃의 크기를 고려하지 않은 `3-1.ipynb`의 단순 CF 알고리즘(`CF_simple()`, 1.024)보다 약간 개선되었음을 볼 수 있다.

이제 실제 추천을 받는 기능을 구현해보자. 추천을 받을 사용자 ID, 추천 아이템 수, `neighbor_size`를 넘겨주면 이 사용자를 위한 추천 영화 리스트를 넘겨주는 코드이다.

실제 추천을 할 때에는 train/test set을 나눌 필요가 없음에 유의하자.

In [5]:
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
movies.set_index('movie_id', inplace=True)
movies.head()

,title
movie_id,
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)


In [7]:
# 전체 데이터로 full matrix와 cosine similarity 구하기
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

target_user = 2
prefer = rating_matrix.loc[target_user].nlargest(10)
pd.concat({'rating': prefer, 'movie title': movies.loc[prefer.index]['title']}, axis=1)

,rating,movie title
movie_id,,
50,5.0,Star Wars (1977)
100,5.0,Fargo (1996)
127,5.0,"Godfather, The (1972)"
242,5.0,Kolya (1996)
251,5.0,Shall We Dance? (1996)
272,5.0,Good Will Hunting (1997)
275,5.0,Sense and Sensibility (1995)
283,5.0,Emma (1996)
285,5.0,Secrets & Lies (1996)


In [8]:
u_id = rating_matrix.index
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=u_id, columns=u_id)

def recommender(user, n_items=10, neighbor_size=20):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index    # 이미 평가한 영화 확인
    items = rating_matrix.loc[user].drop(rated_index)
    i_id = items.index

    predictions = [cf_knn(user, item, neighbor_size) for item in i_id]   # 예상평점 계산

    recommendations = pd.Series(data=predictions, index=i_id, dtype=float)
    recommendations = recommendations.nlargest(n_items)    # 예상평점이 가장 높은 영화 선택
    recommended_items = movies.loc[recommendations.index]['title']
    return pd.concat({'prediction': recommendations, 'movie title': recommended_items}, axis=1)

recommender(user=target_user, n_items=10, neighbor_size=30)

,prediction,movie title
movie_id,,
1189,5.000000,Prefontaine (1997)
1293,5.000000,Star Kid (1997)
1500,5.000000,Santa with Muscles (1996)
1467,5.000000,"Saint of Fort Washington, The (1993)"
318,4.713836,Schindler's List (1993)
64,4.625273,"Shawshank Redemption, The (1994)"
1594,4.597716,Everest (1998)
1449,4.551697,Pather Panchali (1955)
98,4.540835,"Silence of the Lambs, The (1991)"


## 3.5 최적의 이웃 크기 결정

일반적으로 최적의 이웃 크기가 존재하지만, 구체적으로 얼마가 최적의 크기인지는 분야(domain)에 따라 차이가 있음이 알려져 있다. (저자의 논문: ["Does a one-size recommendation system fit all? the effectiveness of collaborative filtering based recommendation systems across different domains and search modes"](https://dl.acm.org/doi/abs/10.1145/1292591.1292595) ACM Transactions on Information Systems (TOIS) 26.1 (2007): 4-es.) 추천을 하려는 분야가 영화냐 옷이냐에 따라 다르고 같은 영화라 하더라도 고객이 누구인지에 따라서 최적의 이웃 크기가 다르다.

In [9]:
# train set으로 full matrix와 cosine similarity 구하기 
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')

u_id = rating_matrix.index
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=u_id, columns=u_id)

for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print("Neighbor size = %d : RMSE = %.4f" % (neighbor_size, score(cf_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0323
Neighbor size = 20 : RMSE = 1.0192
Neighbor size = 30 : RMSE = 1.0170
Neighbor size = 40 : RMSE = 1.0165
Neighbor size = 50 : RMSE = 1.0167
Neighbor size = 60 : RMSE = 1.0173


RMSE를 최소화하는 최적의 이웃 크기는 대략 40 부근임을 알 수 있다. 하지만 책에서는 30 부근이라는 결과가 나왔는데, 이를 바탕으로 생각해보면 `cf_knn()`을 사용하면 `neighbor_size`는 20 이상이기만 하면 큰 차이가 없다는 것을 알 수 있다.